In [18]:
# %% Imports & Setup
import os, sys, time, random, copy, warnings
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader, Subset, random_split

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import optuna
from optuna.exceptions import TrialPruned

warnings.filterwarnings("ignore")

# Add your utils/configs path if needed
sys.path.append('../Utils')
import configs  # must provide MULTIVIEW_TRAIN_DIR and MULTIVIEW_TEST_DIR (ImageFolder structure)

print("Configs paths:")
print(" TRAIN:", configs.MULTIVIEW_TRAIN_DIR)
print(" TEST :", configs.MULTIVIEW_TEST_DIR)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# reproducible seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


Configs paths:
 TRAIN: C:\Users\lenovo\Desktop\TDDP\outputs\MultiViewData\train
 TEST : C:\Users\lenovo\Desktop\TDDP\outputs\MultiViewData\test
Device: cpu


In [19]:
# %% Model definition (TunableDeepCNN)
class TunableDeepCNN(nn.Module):
    def __init__(self, num_classes=7, base_filters=32, dropout_p=0.5, use_dropout2d=True):
        super(TunableDeepCNN, self).__init__()
        f = base_filters
        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(1, f, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f),
            nn.ReLU(inplace=True),
            nn.Conv2d(f, f, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(f, f*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f*2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f*2, f*2, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f*2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(f*2, f*4, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f*4),
            nn.ReLU(inplace=True),
            nn.Conv2d(f*4, f*4, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(f*4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        # Global Average Pooling -> produces f*4 features
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected
        self.fc1 = nn.Linear(f*4, max(128, f*4))
        self.fc2 = nn.Linear(max(128, f*4), num_classes)

        self.dropout = nn.Dropout(dropout_p)
        self.use_dropout2d = use_dropout2d
        self.dropout2d = nn.Dropout2d(0.3) if use_dropout2d else nn.Identity()

        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.block1(x)
        x = self.dropout2d(x)
        x = self.block2(x)
        x = self.dropout2d(x)
        x = self.block3(x)
        x = self.gap(x)               # [B, C, 1, 1]
        x = x.view(x.size(0), -1)     # [B, C]
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [20]:
# %% Data transforms and splits
base_transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),   # smaller for speed — change to 224 if you have memory
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=12),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

base_transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load full datasets
full_train_dataset = datasets.ImageFolder(configs.MULTIVIEW_TRAIN_DIR, transform=base_transform_train)
full_test_dataset  = datasets.ImageFolder(configs.MULTIVIEW_TEST_DIR, transform=base_transform_test)

NUM_CLASSES = len(full_train_dataset.classes)
print("Classes:", full_train_dataset.classes)
print("NUM_CLASSES:", NUM_CLASSES)

# Create train/validation split (80/20) for Optuna and final training
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# We'll use separate transforms for validation (no augmentation)
# update val_dataset to use test transform (non-augmented)
# random_split returns Subset objects; override dataset attribute transform via a wrapper
class SubsetWithTransform(Subset):
    def __init__(self, subset, transform):
        super().__init__(subset.dataset, subset.indices)
        self.transform = transform

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        path, target = self.dataset.samples[real_idx]
        img = self.dataset.loader(path)
        img = self.transform(img)
        return img, target

val_dataset = SubsetWithTransform(train_dataset, base_transform_test)

# DataLoaders (default batch size used by Optuna objective will override if needed)
DEFAULT_BS = 32
train_loader = DataLoader(train_dataset, batch_size=DEFAULT_BS, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=DEFAULT_BS, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(full_test_dataset, batch_size=DEFAULT_BS, shuffle=False, num_workers=4, pin_memory=True)


Classes: ['Ash', 'Beech', 'Douglas Fir', 'Oak', 'Pine', 'Red Oak', 'Spruce']
NUM_CLASSES: 7


In [21]:
# %% training/validation helpers (return history)
def train_one_epoch(model, dataloader, criterion, optimizer, device, use_mixup=False, mixup_alpha=0.4):
    model.train()
    running_loss, running_corrects, total = 0.0, 0, 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total += inputs.size(0)
    return running_loss / total, running_corrects.double() / total

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, running_corrects, total = 0.0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += inputs.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return running_loss / total, running_corrects.double() / total, all_labels, all_preds

def train_model_with_history(model, train_loader, val_loader, criterion, optimizer, scheduler=None,
                             num_epochs=8, device=device, patience=3):
    history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
    best_val_acc = 0.0
    best_weights = copy.deepcopy(model.state_dict())
    patience_counter = 0

    for epoch in range(num_epochs):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)

        if scheduler is not None:
            # scheduler can be ReduceLROnPlateau or other; if plateau requires val_loss, call step accordingly
            try:
                scheduler.step(val_loss)
            except TypeError:
                # call without args
                scheduler.step()

        history["train_loss"].append(float(tr_loss))
        history["train_acc"].append(float(tr_acc))
        history["val_loss"].append(float(val_loss))
        history["val_acc"].append(float(val_acc))

        # early save best
        if val_acc > best_val_acc:
            best_val_acc = float(val_acc)
            best_weights = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    model.load_state_dict(best_weights)
    return model, history


In [ ]:
# %% Optuna objective (fast; uses subset of train set)
def objective(trial):
    # hyperparams
    base_filters = trial.suggest_categorical("base_filters", [16, 32, 48])
    dropout_p = trial.suggest_uniform("dropout_p", 0.2, 0.6)
    use_dropout2d = trial.suggest_categorical("use_dropout2d", [True, False])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-7, 1e-3)
    batch_size = trial.suggest_categorical("batch_size", [16, 32])

    # Create a small subset of training data for fast trials
    small_frac = 0.25
    n_small = max(32, int(small_frac * len(train_dataset)))
    small_indices = np.random.choice(range(len(train_dataset)), size=n_small, replace=False)
    small_subset = Subset(train_dataset, small_indices)
    small_loader = DataLoader(small_subset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

    # validation loader: use val_loader defined earlier but maybe smaller batch
    val_loader_small = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

    # model + criterion + optimizer
    model = TunableDeepCNN(num_classes=NUM_CLASSES, base_filters=base_filters, dropout_p=dropout_p, use_dropout2d=use_dropout2d).to(device)

    # compute class weights on small subset (robust to imbalance)
    labels_small = []
    for _, lab in small_subset:
        labels_small.append(int(lab))
    classes = np.unique(labels_small)
    if len(classes) == 0:
        class_weights = torch.ones(NUM_CLASSES, device=device)
    else:
        cw = compute_class_weight('balanced', classes=np.unique(labels_small), y=labels_small)
        class_weights = torch.tensor(cw, dtype=torch.float, device=device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5)

    # train short with small patience and report intermediate results for pruning
    best_val = 0.0
    for epoch in range(1, 9):  # short: 8 epochs
        model.train()
        for X, y in small_loader:
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            logits = model(X)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

        # evaluate on validation (full val_dataset but small batch)
        model.eval()
        val_loss, val_acc, _, _ = validate(model, val_loader_small, criterion, device)
        trial.report(float(val_acc), epoch)

        # pruning check
        if trial.should_prune():
            raise TrialPruned()

        if val_acc > best_val:
            best_val = float(val_acc)

    return best_val

# Run a quick study first
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=6, timeout=None, show_progress_bar=True)
print("Fast study best:", study.best_trial.params, "val_acc=", study.best_value)


[I 2025-09-11 12:47:03,686] A new study created in memory with name: no-name-577778c5-db16-4754-88e8-1edd60fdc710


  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# %% Retrain final best model on full train dataset (longer) then evaluate on test set
best_params = study.best_trial.params
print("Best params to retrain:", best_params)

# create dataloaders for full training with chosen batch size
final_bs = int(best_params.get("batch_size", 32))
full_train_loader = DataLoader(train_dataset, batch_size=final_bs, shuffle=True, num_workers=4, pin_memory=True)
final_val_loader = DataLoader(val_dataset, batch_size=final_bs, shuffle=False, num_workers=4, pin_memory=True)
final_test_loader = DataLoader(full_test_dataset, batch_size=final_bs, shuffle=False, num_workers=4, pin_memory=True)

# build model with best params
model_final = TunableDeepCNN(num_classes=NUM_CLASSES,
                            base_filters=int(best_params.get("base_filters", 32)),
                            dropout_p=float(best_params.get("dropout_p", 0.5)),
                            use_dropout2d=bool(best_params.get("use_dropout2d", True))).to(device)

# class weights computed on entire train_dataset
train_labels = [y for _, y in train_dataset]
if len(train_labels) == 0:
    cw = np.ones(NUM_CLASSES)
else:
    cw = compute_class_weight('balanced', classes=np.arange(NUM_CLASSES), y=train_labels)
class_weights = torch.tensor(cw, dtype=torch.float, device=device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model_final.parameters(), lr=float(best_params.get("lr", 1e-4)), weight_decay=float(best_params.get("weight_decay", 1e-6)))
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=4, factor=0.5)

# Train for more epochs (final)
final_epochs = 30
model_final, history_final = train_model_with_history(model_final, full_train_loader, final_val_loader,
                                                      criterion, optimizer, scheduler,
                                                      num_epochs=final_epochs, device=device, patience=6)

# Save best final model
out_path = Path("best_final_model.pth")
torch.save(model_final.state_dict(), out_path)
print("Saved final model to:", out_path)

# Evaluate on test set
test_loss, test_acc, test_labels, test_preds = validate(model_final, final_test_loader, criterion, device)
print(f"Test Acc: {test_acc:.4f}, Test Loss: {test_loss:.4f}")
print("\nClassification Report:\n", classification_report(test_labels, test_preds, target_names=full_train_dataset.classes))
cm = confusion_matrix(test_labels, test_preds)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=full_train_dataset.classes, yticklabels=full_train_dataset.classes, cmap="Blues")
plt.ylabel("True"); plt.xlabel("Predicted"); plt.title("Final Confusion Matrix")
plt.show()

# Plot learning curves
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_final["train_loss"], label="train_loss")
plt.plot(history_final["val_loss"], label="val_loss")
plt.legend(); plt.title("Loss")
plt.subplot(1,2,2)
plt.plot(history_final["train_acc"], label="train_acc")
plt.plot(history_final["val_acc"], label="val_acc")
plt.legend(); plt.title("Accuracy")
plt.show()


[I 2025-09-11 12:39:41,660] A new study created in memory with name: no-name-bba37c50-e95e-41f7-9593-7748ca4f93f5


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2025-09-11 12:39:41,689] Trial 0 failed with parameters: {'lr': 0.000725780656152857, 'dropout': 0.37567359902864556, 'weight_decay': 0.00017679551481686453} because of the following error: NameError("name 'train_subset' is not defined").
Traceback (most recent call last):
  File "c:\Users\lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_72140\3638115026.py", line 15, in objective
    subset_size = int(0.3 * len(train_subset))
                                ^^^^^^^^^^^^
NameError: name 'train_subset' is not defined
[W 2025-09-11 12:39:41,691] Trial 0 failed with value None.


NameError: name 'train_subset' is not defined